In [ ]:
!pip install  peft accelerate transformers flask pyngrok bitsandbytes

In [ ]:
!pip install flask-cors

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("NGROK_TOKEN")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
from pyngrok import ngrok

import torch
from flask import Flask, request, jsonify
from flask_cors import CORS

app =  Flask(__name__)
CORS(app)




tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct",device_map="auto",load_in_4bit=True)
model = PeftModel.from_pretrained(base_model,"abhi317/final_llama")

model.generation_config.cache_implementation="static"

compiled_model = torch.compile(model,mode="reduce-overhead",fullgraph=True)

In [ ]:
def answer_question(question, compiled_model, tokenizer):

  tokenizer.pad_token = tokenizer.eos_token

  input_ids = tokenizer(question,return_tensors="pt").to("cuda")
  attention_mask = input_ids['attention_mask'].to("cuda")

  output = compiled_model.generate(
      input_ids['input_ids'],
      max_length=1024,
      num_beams=1,
      attention_mask=attention_mask,
      temperature=0.4,
      top_k=50,
      top_p=0.95,
      do_sample=True,
      early_stopping=False,
  )
  response = tokenizer.decode(output[0], skip_special_tokens=True,)
  x = len(question)
  answer = response[x:].strip()
  return answer


@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    print(data)
    question = data['text']
    system_prompt = "You are CHAT-WITH-ML, the virtual assistant specializing in machine learning and artificial intelligence. Your role is to provide expert guidance, answer questions in simple format, and offer assistance to users navigating the complex world of ML. Your tone should be informative, patient, and technically proficient, aiming to empower users with knowledge and understanding ..."
    questions = system_prompt+question
    answer = answer_question(questions, compiled_model, tokenizer)
    answer=answer.replace(questions," ")
    return jsonify({'prediction': answer})

if __name__ == '__main__':
    # Start ngrok
    public_url = ngrok.connect(5000)
    print(f' * ngrok tunnel "{public_url}" -> "http://127.0.0.1:5000"')

    # Run Flask app
    app.run()